# Federal Reserve ETL Pipeline - Interactive Notebook

This notebook provides an interactive interface for the Federal Reserve ETL Pipeline, allowing you to extract data from FRED and Haver Analytics APIs with real-time feedback and visualization.

## Features
- ✅ FRED API integration with rate limiting
- ✅ Haver Analytics API integration
- ✅ Factory pattern for data source creation
- ✅ Comprehensive error handling
- ✅ Metadata retrieval and data validation
- ✅ Export to CSV, JSON, and Excel formats
- ✅ Interactive data visualization

## Prerequisites
- FRED_API_KEY environment variable set
- Optional: HAVER_API_KEY for Haver Analytics
- Network connectivity to API endpoints

## 1. Setup and Imports

In [1]:
# Core imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
sys.path.insert(0, str(Path.cwd() / 'src'))

# Federal Reserve ETL imports
from federal_reserve_etl import (
    create_data_source,
    FREDDataSource,
    HaverDataSource,
    get_config_manager,
    validate_source_credentials
)
from federal_reserve_etl.utils import (
    ConnectionError,
    AuthenticationError,
    DataRetrievalError,
    ValidationError,
    setup_logging
)

# Setup logging for notebook
logger = setup_logging(log_level="INFO", enable_console=True)

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

print("✅ Federal Reserve ETL Pipeline - Interactive Notebook Initialized")
print(f"📁 Working Directory: {Path.cwd()}")
print(f"🐍 Python Version: {sys.version.split()[0]}")
print(f"🐼 Pandas Version: {pd.__version__}")

ModuleNotFoundError: No module named 'matplotlib'

## 2. API Key Configuration and Validation

In [ ]:
# Check API key configuration
fred_api_key = os.getenv('FRED_API_KEY')
haver_api_key = os.getenv('HAVER_API_KEY')

print("🔑 API Key Status:")
print(f"   FRED API Key: {'✅ Set' if fred_api_key else '❌ Not Set'}")
if fred_api_key:
    print(f"   FRED Key Length: {len(fred_api_key)} characters (expected: 32)")
print(f"   Haver API Key: {'✅ Set' if haver_api_key else '❌ Not Set (Optional)'}")

# Validate credentials
if fred_api_key:
    try:
        fred_valid = validate_source_credentials('fred')
        print(f"\n🔍 FRED Credential Validation: {'✅ Valid' if fred_valid else '❌ Invalid'}")
    except Exception as e:
        print(f"\n⚠️ FRED Credential Validation Failed: {e}")
else:
    print("\n⚠️ Please set FRED_API_KEY environment variable to continue")
    print("   Example: os.environ['FRED_API_KEY'] = 'your_api_key_here'")

## 3. Quick Start - FRED Data Extraction

In [ ]:
# Quick demonstration with Federal Funds Rate
if fred_api_key:
    print("🚀 Quick Start: Extracting Federal Funds Rate (Last 30 Days)")
    
    # Calculate date range
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    try:
        # Create FRED data source and extract data
        with create_data_source('fred', api_key=fred_api_key) as fred:
            print(f"📊 Extracting FEDFUNDS from {start_date} to {end_date}")
            
            df = fred.get_data(
                variables="FEDFUNDS",
                start_date=start_date,
                end_date=end_date
            )
            
            print(f"\n✅ Successfully extracted {len(df)} observations")
            print(f"📈 Data range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
            
            # Display first few rows
            print("\n📋 Sample Data:")
            print(df.head().round(2))
            
            # Quick visualization
            plt.figure(figsize=(10, 4))
            plt.plot(df.index, df['FEDFUNDS'], marker='o', linewidth=2, markersize=4)
            plt.title('Federal Funds Rate - Last 30 Days', fontsize=14, fontweight='bold')
            plt.xlabel('Date')
            plt.ylabel('Rate (%)')
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
    except Exception as e:
        print(f"❌ Error during quick start: {e}")
        print("💡 This might be due to weekends/holidays - try a longer date range")
else:
    print("⚠️ FRED API Key required for quick start demonstration")

## 4. Data Source Factory and Configuration

In [ ]:
# Demonstrate data source factory pattern
print("🏭 Data Source Factory Demonstration")

# Get configuration manager
config_mgr = get_config_manager()

print("\n📋 Available Data Sources:")
sources = ['fred', 'haver']

for source in sources:
    try:
        # Test credential validation
        is_valid = config_mgr.validate_credentials(source)
        status = "✅ Valid" if is_valid else "❌ Invalid"
        
        # Get missing credentials if any
        missing = config_mgr.get_missing_credentials(source)
        missing_str = f" (Missing: {missing})" if missing else ""
        
        print(f"   {source.upper()}: {status}{missing_str}")
        
        if is_valid:
            # Get data source configuration
            source_config = config_mgr.get_data_source_config(source)
            print(f"      Rate Limit: {source_config.get('rate_limit', 'Not specified')} requests/minute")
            
            # Test factory creation
            if source == 'fred' and fred_api_key:
                ds = create_data_source(source, api_key=fred_api_key)
                print(f"      Factory Test: ✅ Created {type(ds).__name__}")
            elif source == 'haver' and haver_api_key:
                ds = create_data_source(source, api_key=haver_api_key)
                print(f"      Factory Test: ✅ Created {type(ds).__name__}")
                
    except Exception as e:
        print(f"   {source.upper()}: ❌ Error - {e}")

## 5. Interactive Data Extraction

In [ ]:
# Interactive data extraction with user-friendly interface
def extract_economic_data(variables, start_date, end_date, source='fred', api_key=None):
    """
    Extract economic data with comprehensive error handling and reporting
    
    Args:
        variables: Single variable (str) or list of variables
        start_date: Start date (YYYY-MM-DD format or datetime)
        end_date: End date (YYYY-MM-DD format or datetime)
        source: Data source ('fred' or 'haver')
        api_key: API key for the specified source
    
    Returns:
        tuple: (DataFrame, metadata_dict)
    """
    
    # Convert single variable to list for consistent processing
    if isinstance(variables, str):
        variables = [variables]
    
    print(f"🔄 Starting Data Extraction")
    print(f"   Source: {source.upper()}")
    print(f"   Variables: {variables}")
    print(f"   Date Range: {start_date} to {end_date}")
    
    try:
        # Create data source
        with create_data_source(source, api_key=api_key) as ds:
            print(f"\n✅ Connected to {source.upper()} API")
            
            # Extract data
            df = ds.get_data(
                variables=variables,
                start_date=start_date,
                end_date=end_date
            )
            
            print(f"📊 Extracted {len(df)} observations")
            
            # Get metadata
            metadata = ds.get_metadata(variables)
            print(f"📋 Retrieved metadata for {len(metadata)} variables")
            
            # Display summary statistics
            print(f"\n📈 Data Summary:")
            print(f"   Date Range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
            print(f"   Observations: {len(df)}")
            print(f"   Variables: {list(df.columns)}")
            
            # Check for missing data
            missing_data = df.isnull().sum()
            if missing_data.sum() > 0:
                print(f"\n⚠️ Missing Data:")
                for var, missing in missing_data.items():
                    if missing > 0:
                        pct = (missing / len(df)) * 100
                        print(f"   {var}: {missing} observations ({pct:.1f}%)")
            else:
                print(f"\n✅ No missing data detected")
            
            return df, metadata
            
    except Exception as e:
        print(f"❌ Extraction failed: {e}")
        print(f"💡 Error type: {type(e).__name__}")
        raise

print("✅ Interactive extraction function loaded")
print("💡 Use extract_economic_data(variables, start_date, end_date, source, api_key) to extract data")

## 6. Example: Multiple Economic Indicators

In [ ]:
# Extract multiple key economic indicators
if fred_api_key:
    print("📊 Extracting Key Economic Indicators for 2023")
    
    # Define variables and date range
    key_indicators = ["FEDFUNDS", "DGS10", "TB3MS", "UNRATE", "CPIAUCSL"]
    start_date = "2023-01-01"
    end_date = "2023-12-31"
    
    try:
        # Extract data
        df, metadata = extract_economic_data(
            variables=key_indicators,
            start_date=start_date,
            end_date=end_date,
            source='fred',
            api_key=fred_api_key
        )
        
        # Display metadata
        print("\n📋 Variable Descriptions:")
        for var, meta in metadata.items():
            print(f"   {var}: {meta.get('name', 'No description')}")
        
        # Display sample data
        print("\n📋 Sample Data (First 10 Rows):")
        print(df.head(10).round(2))
        
        # Create visualization
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.ravel()
        
        for i, col in enumerate(df.columns):
            if i < len(axes):
                # Plot data with proper handling of missing values
                data_to_plot = df[col].dropna()
                if len(data_to_plot) > 0:
                    axes[i].plot(data_to_plot.index, data_to_plot.values, 
                               linewidth=2, marker='o', markersize=2)
                    axes[i].set_title(f'{col}\n{metadata[col].get("name", "")[:50]}', 
                                    fontsize=10, fontweight='bold')
                    axes[i].grid(True, alpha=0.3)
                    axes[i].tick_params(axis='x', rotation=45)
                else:
                    axes[i].text(0.5, 0.5, f'No data for {col}', 
                               ha='center', va='center', transform=axes[i].transAxes)
        
        # Remove empty subplot
        if len(df.columns) < len(axes):
            fig.delaxes(axes[-1])
        
        plt.suptitle('Key Economic Indicators - 2023', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Store data for later use
        economic_indicators_2023 = df
        economic_metadata = metadata
        
    except Exception as e:
        print(f"❌ Failed to extract economic indicators: {e}")
else:
    print("⚠️ FRED API Key required for economic indicators demonstration")

## 7. Data Export Capabilities

In [ ]:
# Demonstrate various export formats
def export_data(df, metadata, base_filename="economic_data", formats=['csv', 'json', 'excel']):
    """
    Export data to multiple formats with metadata
    
    Args:
        df: DataFrame to export
        metadata: Metadata dictionary
        base_filename: Base name for output files
        formats: List of formats to export ('csv', 'json', 'excel')
    
    Returns:
        dict: Paths of created files
    """
    
    output_dir = Path.cwd() / "output"
    output_dir.mkdir(exist_ok=True)
    
    exported_files = {}
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"📁 Exporting to: {output_dir}")
    
    try:
        # CSV Export
        if 'csv' in formats:
            csv_path = output_dir / f"{base_filename}_{timestamp}.csv"
            df.to_csv(csv_path, index=True)
            exported_files['csv'] = csv_path
            print(f"✅ CSV exported: {csv_path.name} ({csv_path.stat().st_size:,} bytes)")
        
        # JSON Export with metadata
        if 'json' in formats:
            json_path = output_dir / f"{base_filename}_{timestamp}.json"
            export_data = {
                'data': df.reset_index().to_dict('records'),
                'metadata': metadata,
                'export_info': {
                    'timestamp': datetime.now().isoformat(),
                    'observations': len(df),
                    'variables': list(df.columns),
                    'date_range': {
                        'start': df.index.min().isoformat() if len(df) > 0 else None,
                        'end': df.index.max().isoformat() if len(df) > 0 else None
                    }
                }
            }
            
            import json
            with open(json_path, 'w') as f:
                json.dump(export_data, f, indent=2, default=str)
            
            exported_files['json'] = json_path
            print(f"✅ JSON exported: {json_path.name} ({json_path.stat().st_size:,} bytes)")
        
        # Excel Export with multiple sheets
        if 'excel' in formats:
            excel_path = output_dir / f"{base_filename}_{timestamp}.xlsx"
            
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                # Data sheet
                df.to_excel(writer, sheet_name='Data', index=True)
                
                # Metadata sheet
                if metadata:
                    meta_df = pd.DataFrame.from_dict(metadata, orient='index')
                    meta_df.to_excel(writer, sheet_name='Metadata', index=True)
                
                # Summary sheet
                summary_data = {
                    'Metric': ['Variables', 'Observations', 'Date Range Start', 'Date Range End', 'Export Date'],
                    'Value': [
                        len(df.columns),
                        len(df),
                        df.index.min().strftime('%Y-%m-%d') if len(df) > 0 else 'N/A',
                        df.index.max().strftime('%Y-%m-%d') if len(df) > 0 else 'N/A',
                        datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    ]
                }
                summary_df = pd.DataFrame(summary_data)
                summary_df.to_excel(writer, sheet_name='Summary', index=False)
            
            exported_files['excel'] = excel_path
            print(f"✅ Excel exported: {excel_path.name} ({excel_path.stat().st_size:,} bytes)")
        
        return exported_files
        
    except Exception as e:
        print(f"❌ Export failed: {e}")
        return {}

# Test export if we have data
if 'economic_indicators_2023' in locals() and len(economic_indicators_2023) > 0:
    print("💾 Testing Data Export...")
    exported = export_data(
        df=economic_indicators_2023.head(50),  # Export first 50 rows for demo
        metadata=economic_metadata,
        base_filename="demo_economic_indicators",
        formats=['csv', 'json']  # Skip Excel for demo
    )
    
    if exported:
        print(f"\n📊 Export Summary:")
        for format_type, file_path in exported.items():
            print(f"   {format_type.upper()}: {file_path}")
else:
    print("⚠️ No data available for export demonstration")
    print("💡 Run the previous cells to extract data first")

## 8. Error Handling and Recovery

In [ ]:
# Demonstrate comprehensive error handling
def demonstrate_error_handling():
    """
    Demonstrate various error scenarios and recovery patterns
    """
    
    print("🔍 Error Handling Demonstration\n")
    
    # Test 1: Invalid API Key
    print("Test 1: Invalid API Key")
    try:
        invalid_key = '1234567890123456789012345678901X'  # 32 chars but invalid
        with create_data_source('fred', api_key=invalid_key) as fred:
            df = fred.get_data(variables="FEDFUNDS", start_date="2023-01-01", end_date="2023-01-31")
        print("❌ This should have failed!")
    except AuthenticationError as e:
        print(f"✅ Correctly caught AuthenticationError: {e}")
    except Exception as e:
        print(f"⚠️ Caught different error: {type(e).__name__}: {e}")
    
    print("\n" + "-"*50 + "\n")
    
    # Test 2: Invalid Variable Code
    if fred_api_key:
        print("Test 2: Invalid Variable Code")
        try:
            with create_data_source('fred', api_key=fred_api_key) as fred:
                df = fred.get_data(
                    variables="INVALID_VARIABLE_CODE_12345",
                    start_date="2023-01-01",
                    end_date="2023-01-31"
                )
            print("❌ This should have failed!")
        except DataRetrievalError as e:
            print(f"✅ Correctly caught DataRetrievalError: {e}")
        except Exception as e:
            print(f"⚠️ Caught different error: {type(e).__name__}: {e}")
    
    print("\n" + "-"*50 + "\n")
    
    # Test 3: Invalid Date Range
    if fred_api_key:
        print("Test 3: Invalid Date Range (Start > End)")
        try:
            with create_data_source('fred', api_key=fred_api_key) as fred:
                df = fred.get_data(
                    variables="FEDFUNDS",
                    start_date="2023-12-31",  # Start after end
                    end_date="2023-01-01"
                )
            print("❌ This should have failed!")
        except ValidationError as e:
            print(f"✅ Correctly caught ValidationError: {e}")
        except Exception as e:
            print(f"⚠️ Caught different error: {type(e).__name__}: {e}")
    
    print("\n" + "-"*50 + "\n")
    
    # Test 4: Future Date Range
    if fred_api_key:
        print("Test 4: Future Date Range (No Data Available)")
        try:
            future_start = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
            future_end = (datetime.now() + timedelta(days=60)).strftime('%Y-%m-%d')
            
            with create_data_source('fred', api_key=fred_api_key) as fred:
                df = fred.get_data(
                    variables="FEDFUNDS",
                    start_date=future_start,
                    end_date=future_end
                )
            print(f"⚠️ Unexpectedly succeeded - got {len(df)} rows")
        except DataRetrievalError as e:
            print(f"✅ Correctly caught DataRetrievalError: {e}")
        except Exception as e:
            print(f"⚠️ Caught different error: {type(e).__name__}: {e}")
    
    print("\n✅ Error handling demonstration complete")

# Run error handling demonstration
demonstrate_error_handling()

## 9. Advanced Features

In [ ]:
# Advanced features demonstration
def create_economic_dashboard(variables, start_date, end_date):
    """
    Create a comprehensive economic data dashboard
    
    Args:
        variables: List of FRED variable codes
        start_date: Start date for data extraction
        end_date: End date for data extraction
    """
    
    if not fred_api_key:
        print("⚠️ FRED API Key required for dashboard")
        return
    
    print(f"📊 Creating Economic Dashboard")
    print(f"   Variables: {variables}")
    print(f"   Period: {start_date} to {end_date}")
    
    try:
        # Extract data with progress reporting
        df, metadata = extract_economic_data(
            variables=variables,
            start_date=start_date,
            end_date=end_date,
            source='fred',
            api_key=fred_api_key
        )
        
        # Create comprehensive dashboard
        fig = plt.figure(figsize=(20, 12))
        
        # Time series plots
        for i, col in enumerate(df.columns[:4]):  # Limit to first 4 variables
            plt.subplot(2, 3, i+1)
            data = df[col].dropna()
            if len(data) > 0:
                plt.plot(data.index, data.values, linewidth=2, color=f'C{i}')
                plt.title(f'{col}\n{metadata[col].get("name", "")[:40]}...', 
                         fontsize=12, fontweight='bold')
                plt.grid(True, alpha=0.3)
                plt.xticks(rotation=45)
                
                # Add trend line
                x_numeric = np.arange(len(data))
                z = np.polyfit(x_numeric, data.values, 1)
                p = np.poly1d(z)
                plt.plot(data.index, p(x_numeric), "--", alpha=0.7, color='red')
        
        # Correlation heatmap
        plt.subplot(2, 3, 5)
        correlation_matrix = df.corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                   square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
        plt.title('Variable Correlations', fontsize=12, fontweight='bold')
        
        # Summary statistics
        plt.subplot(2, 3, 6)
        plt.axis('off')
        
        # Create summary text
        summary_text = "📊 Dashboard Summary\n\n"
        summary_text += f"📅 Period: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}\n"
        summary_text += f"📈 Observations: {len(df):,}\n"
        summary_text += f"📊 Variables: {len(df.columns)}\n\n"
        
        # Variable statistics
        for col in df.columns:
            data = df[col].dropna()
            if len(data) > 0:
                mean_val = data.mean()
                std_val = data.std()
                summary_text += f"{col}: μ={mean_val:.2f}, σ={std_val:.2f}\n"
        
        plt.text(0.05, 0.95, summary_text, transform=plt.gca().transAxes,
                fontsize=10, verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
        
        plt.suptitle('Federal Reserve Economic Data Dashboard', 
                    fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.show()
        
        return df, metadata
        
    except Exception as e:
        print(f"❌ Dashboard creation failed: {e}")
        return None, None

# Create sample dashboard
if fred_api_key:
    dashboard_variables = ["FEDFUNDS", "DGS10", "UNRATE", "CPIAUCSL"]
    dashboard_data, dashboard_meta = create_economic_dashboard(
        variables=dashboard_variables,
        start_date="2020-01-01",
        end_date="2023-12-31"
    )
else:
    print("⚠️ FRED API Key required for dashboard demonstration")

## 10. Custom Analysis Functions

In [ ]:
# Custom analysis functions for economic data
def analyze_rate_changes(df, rate_column):
    """
    Analyze rate changes and identify significant movements
    
    Args:
        df: DataFrame containing rate data
        rate_column: Column name containing rate data
    
    Returns:
        dict: Analysis results
    """
    
    if rate_column not in df.columns:
        print(f"❌ Column '{rate_column}' not found in DataFrame")
        return {}
    
    data = df[rate_column].dropna()
    if len(data) < 2:
        print(f"❌ Insufficient data for analysis")
        return {}
    
    # Calculate changes
    changes = data.diff().dropna()
    
    # Analysis results
    analysis = {
        'total_observations': len(data),
        'current_rate': data.iloc[-1],
        'period_start_rate': data.iloc[0],
        'total_change': data.iloc[-1] - data.iloc[0],
        'max_rate': data.max(),
        'min_rate': data.min(),
        'average_rate': data.mean(),
        'volatility': data.std(),
        'largest_increase': changes.max(),
        'largest_decrease': changes.min(),
        'increase_dates': changes[changes > 0].index.tolist(),
        'decrease_dates': changes[changes < 0].index.tolist(),
        'no_change_dates': changes[changes == 0].index.tolist()
    }
    
    # Print analysis
    print(f"📊 Rate Analysis for {rate_column}")
    print(f"   Period: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
    print(f"   Observations: {analysis['total_observations']:,}")
    print(f"\n📈 Rate Statistics:")
    print(f"   Current Rate: {analysis['current_rate']:.2f}%")
    print(f"   Starting Rate: {analysis['period_start_rate']:.2f}%")
    print(f"   Total Change: {analysis['total_change']:+.2f}%")
    print(f"   Range: {analysis['min_rate']:.2f}% to {analysis['max_rate']:.2f}%")
    print(f"   Average: {analysis['average_rate']:.2f}%")
    print(f"   Volatility (σ): {analysis['volatility']:.2f}%")
    
    print(f"\n🔄 Change Analysis:")
    print(f"   Largest Increase: +{analysis['largest_increase']:.2f}%")
    print(f"   Largest Decrease: {analysis['largest_decrease']:.2f}%")
    print(f"   Rate Increases: {len(analysis['increase_dates'])} occasions")
    print(f"   Rate Decreases: {len(analysis['decrease_dates'])} occasions")
    print(f"   No Changes: {len(analysis['no_change_dates'])} occasions")
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Rate over time
    ax1.plot(data.index, data.values, linewidth=2, color='blue', label=rate_column)
    ax1.axhline(y=analysis['average_rate'], color='red', linestyle='--', 
               alpha=0.7, label=f'Average ({analysis["average_rate"]:.2f}%)')
    ax1.fill_between(data.index, analysis['min_rate'], analysis['max_rate'], 
                    alpha=0.1, color='gray', label='Range')
    ax1.set_title(f'{rate_column} Over Time', fontweight='bold')
    ax1.set_ylabel('Rate (%)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Rate changes
    ax2.bar(changes.index, changes.values, width=1, 
           color=['green' if x > 0 else 'red' if x < 0 else 'gray' for x in changes.values])
    ax2.axhline(y=0, color='black', linewidth=0.5)
    ax2.set_title(f'{rate_column} Changes', fontweight='bold')
    ax2.set_ylabel('Change (%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return analysis

# Test analysis function
if 'economic_indicators_2023' in locals() and 'FEDFUNDS' in economic_indicators_2023.columns:
    print("🔍 Analyzing Federal Funds Rate Changes...\n")
    fed_funds_analysis = analyze_rate_changes(economic_indicators_2023, 'FEDFUNDS')
else:
    print("⚠️ Federal Funds Rate data not available for analysis")
    print("💡 Run the economic indicators extraction cell first")

## 11. Quick Commands Reference

In [ ]:
# Quick commands and utilities
def show_available_functions():
    """
    Display available functions and their usage
    """
    
    functions = {
        'extract_economic_data()': 'Extract data from FRED or Haver APIs',
        'export_data()': 'Export DataFrame to CSV, JSON, or Excel',
        'create_economic_dashboard()': 'Create comprehensive data dashboard',
        'analyze_rate_changes()': 'Analyze rate changes and movements',
        'validate_source_credentials()': 'Validate API credentials',
        'create_data_source()': 'Create FRED or Haver data source',
        'get_config_manager()': 'Get configuration manager instance'
    }
    
    print("🛠️ Available Functions:")
    for func, desc in functions.items():
        print(f"   {func:<30} - {desc}")
    
    print(f"\n🔑 Required Environment Variables:")
    print(f"   FRED_API_KEY                   - FRED API key (required)")
    print(f"   HAVER_API_KEY                  - Haver Analytics API key (optional)")
    
    print(f"\n📊 Common Variable Codes:")
    variables = {
        'FEDFUNDS': 'Federal Funds Rate',
        'DGS10': '10-Year Treasury Constant Maturity Rate',
        'TB3MS': '3-Month Treasury Bill Rate',
        'UNRATE': 'Unemployment Rate',
        'CPIAUCSL': 'Consumer Price Index for All Urban Consumers',
        'GDPC1': 'Real Gross Domestic Product',
        'PAYEMS': 'All Employees: Total Nonfarm Payrolls'
    }
    
    for code, desc in variables.items():
        print(f"   {code:<12} - {desc}")
    
    print(f"\n💡 Quick Start Examples:")
    print(f"   # Extract single variable")
    print(f"   df, meta = extract_economic_data('FEDFUNDS', '2023-01-01', '2023-12-31', 'fred', fred_api_key)")
    print(f"   ")
    print(f"   # Export data")
    print(f"   export_data(df, meta, 'my_data', ['csv', 'json'])")
    print(f"   ")
    print(f"   # Analyze rate changes")
    print(f"   analyze_rate_changes(df, 'FEDFUNDS')")

# Show available functions
show_available_functions()

print("\n" + "="*60)
print("🎉 Federal Reserve ETL Pipeline - Interactive Notebook Ready!")
print("="*60)
print("\n💡 To get started:")
print("   1. Ensure FRED_API_KEY environment variable is set")
print("   2. Run the data extraction examples above")
print("   3. Use the analysis and export functions as needed")
print("   4. Create custom visualizations and analysis")
print("\n📚 All functions include comprehensive error handling and logging")
print("🔍 Use help(function_name) for detailed documentation")
